In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Originally-originaly-originally
# many time originally it is fork from
# https://www.kaggle.com/matthewa313/ensembling-with-logistic-regression-lb-81-7

import pandas as pd
import numpy  as np
from scipy.special import expit, logit
 
almost_zero = 1e-5 # 0.00001
almost_one  = 1-almost_zero # 0.99999

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

scores = {}

# Gated Recurrent Unit + Convolutional Neural Network
df = pd.read_csv("NLP.csv",index_col="id").rename(columns={'project_is_approved': 'NLP'}) # 0.80121
scores["NLP"] = 0.7959

# "The Choice is Yours"
# df["choice"] = pd.read_csv("../input/the-choice-is-yours/blend_submission.csv")['project_is_approved'].values # 0.793
# scores["choice"] = 0.793

# LightGBM and TFIDF
df["Lgbm_tf"] = pd.read_csv("pred_LGB_LR_005.csv")['project_is_approved'].values # 0.7947
scores["Lgbm_tf"] = 0.80569


# Gated Reccurrent Unit + ATT with LGBM, TFIDF + EDA
df["grnn_Att_lgbm"] = pd.read_csv("GRU.csv")['project_is_approved'].values # 0.81177
scores["grnn_Att_lgbm"] = 0.81177

# Capsule Networks
df["capnet"] = pd.read_csv("Capsule.csv")['project_is_approved'].values # 0.79590
scores["capnet"] = 0.79590


# PoWER 64 #best
df["LogReg"] = pd.read_csv("pred_LogReg.csv")['project_is_approved'].values # 0.81829
scores["LogReg"] = 0.79759

# More NN..
# Add https://www.kaggle.com/emotionevil/nlp-and-stacking-starter-dpcnn-lgb-lb0-80/notebook

weights = [0] * len(df.columns)
power = 90
dic = {}

for i,col in enumerate(df.columns):
    weights[i] = scores[col] ** power
    dic[i] = df[col].clip(almost_zero,almost_one).apply(logit) * weights[i] 
    
print(weights[:])
totalweight = sum(weights)

temp = []
for x in dic:
    if x == 0:
        temp = dic[x]
    else:
        temp = temp+dic[x]

temp = temp/(totalweight)

df["project_is_approved"] = temp
df["project_is_approved"] = df["project_is_approved"].apply(expit)
df["project_is_approved"].to_csv("ensembling_submission_0422.csv", index=True, header=True)
print(df["project_is_approved"].head())

[1.1947180446167059e-09, 3.590191382698521e-09, 7.062683801881151e-09, 1.1947180446167059e-09, 1.4460157482908962e-09]
id
p233245    0.886417
p096795    0.856727
p236235    0.856272
p233680    0.846898
p171879    0.865223
Name: project_is_approved, dtype: float64
